\
            # 38. 协程：从生成器到 async/await（Coroutines: Generators to async/await）

            用最小例子理解：
- 生成器如何暂停/恢复（next/send）
- `yield from` 如何把多个生成器组合成流水线
- `async def/await` 解决了什么问题、与线程有什么区别

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 迭代器/生成器（yield）基础
- 函数/异常
- with 语句


## 知识点地图

- 1. 生成器是“可暂停的函数”
- 2. yield from：把“子生成器”拼起来
- 3. async/await：把“可暂停”用于 I/O 并发
- 4. 协程不是线程：并发来自“主动让出控制权”


## 自检清单（学完打勾）

- [ ] 理解生成器的暂停/恢复与 next/send
- [ ] 知道 yield from 的作用（委托子生成器）
- [ ] 知道 coroutine/awaitable 的基本含义
- [ ] 能写出 async/await 的最小示例并解释执行模型


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：生成器是“可暂停的函数”

- `yield` 让函数暂停并产出一个值；下次 `next()` 从暂停点继续。
- `send(value)` 可以把数据“送回”生成器内部（高级用法）。
- 生成器适合：流式处理、惰性计算、管道式组合。


In [ ]:
def gen():
    x = yield 1
    yield ('got', x)


g = gen()
print(next(g))
print(g.send(42))


## 知识点 2：yield from：把“子生成器”拼起来

- `yield from subgen()` 会把子生成器产出的值原样向外产出。
- 更重要的是：它会把 `.send/.throw/.close` 协议也一并委托给子生成器。
- 结果：组合生成器更自然，能写出“流水线”式代码。


In [ ]:
def sub():
    yield 1
    yield 2


def outer():
    yield 0
    yield from sub()
    yield 3


print(list(outer()))


## 知识点 3：async/await：把“可暂停”用于 I/O 并发

- `async def` 定义协程函数；调用它返回协程对象（不会立刻执行）。
- `await` 等待一个 awaitable（协程、Task、Future 等）。
- 协程要靠事件循环调度（常见是 `asyncio`）。

提示：
- 在 Jupyter Notebook 里通常支持“顶层 await”。
- 在脚本里通常用 `asyncio.run(main())`（Python 3.7+）。


In [ ]:
import asyncio


async def main():
    print('start')
    await asyncio.sleep(0.1)
    print('after sleep')


await main()


## 知识点 4：协程不是线程：并发来自“主动让出控制权”

- 协程通常运行在单线程：同一时刻只执行一个协程的 Python 代码。
- 并发来自：遇到 `await`（通常是 I/O）时把控制权交回事件循环，让其他协程运行。
- 如果协程里写了很重的 CPU 计算且不 `await`，会阻塞整个事件循环。

经验：CPU 重活交给进程池/线程池；事件循环里只做“协调”。


## 常见坑

- 把 async 当成“自动并发”：没有 await 就不会让出控制权
- 在 async 里直接调用阻塞函数（time.sleep/阻塞 I/O）导致卡住
- 忘记 await：拿到的是协程对象不是结果


## 综合小案例：并发执行 5 个 I/O 任务（模拟）并限制最大并发为 2

目标：把 5 个任务并发执行，但同一时间最多跑 2 个（典型“限流/背压”）。


In [ ]:
import asyncio


async def io_job(i: int):
    await asyncio.sleep(0.05)
    return i * i


async def main():
    sem = asyncio.Semaphore(2)

    async def guarded(i):
        async with sem:
            return await io_job(i)

    results = await asyncio.gather(*[guarded(i) for i in range(5)])
    print(results)


await main()


## 自测题（不写代码也能回答）

- 生成器与协程的“可暂停”有什么相似与不同？
- 为什么说协程适合 I/O 并发而不是 CPU 并行？
- 在 Notebook 里为什么能直接写 await？脚本里通常怎么做？


## 练习题（建议写代码）

- 实现一个生成器流水线：读取文本 -> 分词 -> 过滤 -> 统计。
- 把 io_job 改成：随机失败时重试 2 次（指数退避）。
- 实现一个 async 版本的超时包装器：超过 100ms 返回默认值。
